In [1]:
import numpy as np
import scipy as sc
import numpy.linalg as npl
import scipy.linalg as scl
from scipy.spatial.distance import pdist,squareform
import ase

In [2]:
#%load_ext line_profiler

# Funcs

## get Lennard Jones forces

In [8]:
def get_separations(position,positions):
    
    r = np.subtract(position,positions).reshape((-1,3))
    
    distances2 = np.power(r,2).sum(axis=1)
    norms = np.sqrt(distances2).reshape((-1,1))
    direction_vectors = np.divide(r,norms)
    return distances2,direction_vectors
    
def get_forces_LJ(positions,sigma,epsilon):
    
    ll = list(set(range(Natom)).difference([iatom]))
    distances2,direction_vectors = get_separations(positions[iatom],positions[ll])

    c6 = np.power(np.divide(sigma**2, distances2),3)
    c12 = np.power(c6,2)

    #energy += 0.5 * np.sum(np.multiply(4 * epsilon , np.subtract(c12,c6)))

    forces[iatom] += np.multiply(np.multiply(4 * epsilon ,np.subtract(np.multiply(c12, 12), np.multiply(c6, 6))).reshape((Natom-1,1)), 
                                     direction_vectors).sum(axis=0)
        
    
    return forces

def get_forces_LJ2(positions,sigma,epsilon):
    Natom,_ = positions.shape
    forces = np.zeros((Natom,3))
    
    ll = np.arange(Natom)
    ll = [[ll[i - j] for i in range(1,Natom)] for j in range(Natom, 0, -1)]
    
    pps = np.asarray([positions[ll[it]] for it in range(Natom)],dtype=np.float64)
    
    r = np.subtract(positions.reshape((Natom,1,3)),pps)
    distances2 = np.power(r,2).sum(axis=2)
    norms = np.sqrt(distances2).reshape((Natom,Natom-1,1))
    
    c6 = np.power(np.divide(sigma**2, distances2),3)
    c12 = np.power(c6,2)
    
    forces = np.multiply(np.multiply(4 * epsilon ,np.subtract(np.multiply(c12, 12), np.multiply(c6, 6))).reshape((Natom,Natom-1,1)), 
                                     np.divide(r,norms)).sum(axis=1)
    return forces

#def get_forces_LJ3(positions,sigma,epsilon):
    
n = 10
#f1 = get_forces_LJ(np.random.rand(n,3),sigma=1,epsilon=1)
f2 = get_forces_LJ2(np.random.rand(n,3),sigma=1,epsilon=1)
#np.allclose(f1,f2)

In [9]:
def get_separations(position,positions):
    
    r = np.subtract(position,positions).reshape((-1,3))
    
    distances2 = np.power(r,2).sum(axis=1)
    norms = np.sqrt(distances2).reshape((-1,1))
    direction_vectors = np.divide(r,norms)
    return distances2,direction_vectors
    
def get_forces_LJ(positions,sigma,epsilon):
    Natom,_ = positions.shape
    
    energy = 0
    
    forces = np.zeros((Natom,3))
    
    for iatom in range(Natom):
        ll = list(set(range(Natom)).difference([iatom]))
        distances2,direction_vectors = get_separations(positions[iatom],positions[ll])
        
        c6 = np.power(np.divide(sigma**2, distances2),3)
        c12 = np.power(c6,2)
        
        #energy += 0.5 * np.sum(np.multiply(4 * epsilon , np.subtract(c12,c6)))
        
        forces[iatom] += np.multiply(np.multiply(4 * epsilon ,np.subtract(np.multiply(c12, 12), np.multiply(c6, 6))).reshape((Natom-1,1)), 
                                     direction_vectors).sum(axis=0)
        
    
    return forces

## get spring forces

In [10]:
def get_energy_and_forces_polymer1(positions,r_m,k_spring):
    Natom,_ = positions.shape
    forces = np.zeros(positions.shape)
    energy = 0
    st = range(Natom)
    nd = list(range(1,Natom))+[0]
    for it,jt in zip(st,nd):
        r = positions[it] - positions[jt]
        norm = npl.norm(r) 
        direction_vector = r / norm
        force = - k_spring * (norm - r_m) * direction_vector
        forces[it] += force
        forces[jt] -= force
        energy += 0.5*k_spring * (norm - r_m)**2
        
    return forces

def get_energy_and_forces_polymer2(positions,r_m,k_spring):
    Natom,_ = positions.shape
    forces = np.zeros(positions.shape)
    energy = 0
    pp2 = np.empty(positions.shape)
    pp1 = positions
    pp2[:-1] = positions[1:]
    pp2[-1] = positions[0]
    
    r = np.subtract(pp1,pp2)
    norms = npl.norm(r,axis=1).reshape((-1,1))
    direction_vectors = np.divide(r,norms)
    ffs = - np.multiply(k_spring, np.multiply(np.subtract(norms, r_m), direction_vectors))
    st = range(Natom)
    nd = list(range(1,Natom))+[0]
    for ff,it,jt in zip(ffs,st,nd):
        forces[it] += ff
        forces[jt] -= ff
    return forces

def get_forces_polymer(positions,r_m,k_spring):
    Natom,_ = positions.shape
    forces = np.zeros(positions.shape)
    energy = 0
    pp2 = np.empty(positions.shape)
    pp1 = positions
    pp2[:-1] = positions[1:]
    pp2[-1] = positions[0]
    
    r = np.subtract(pp1,pp2)
    norms = npl.norm(r,axis=1).reshape((-1,1))
    direction_vectors = np.divide(r,norms)
    ffs = - np.multiply(k_spring, np.multiply(np.subtract(norms, r_m), direction_vectors))
    # st = range(Natom)
    # nd = list(range(1,Natom))+[0]
    st = range(Natom-1)
    nd = list(range(1,Natom))
    for ff,it,jt in zip(ffs,st,nd):
        forces[it] += ff
        forces[jt] -= ff
        
        
    return forces

## integrator

In [11]:
def move_velocity_verlet(positions,velocities,accelerations,dt,mass,r_m,epsilon,k_spring):
    
    sigma = r_m / 2**(1./6.)
    
    
    v_half = velocities + 0.5 * dt * accelerations
    new_positions = positions + dt * v_half
    
    forces = np.zeros(positions.shape)
    forces += get_forces_LJ2(new_positions,sigma,epsilon)
    forces += get_forces_polymer(positions,r_m,k_spring)
    
    new_accelerations = forces / mass
    
    new_velocities = v_half + 0.5 * dt * new_accelerations
    
    return new_positions,new_velocities,new_accelerations

## thermostat

In [12]:
def andersen_thermostat(velocities,temperature):  
    velocities = np.random.normal(loc=0.0, scale=np.sqrt(temperature),size=velocities.shape)
    return velocities

## main engine

In [13]:
def MD_NVT_simulator(positions,mass,temperature,r_m,epsilon,k_spring,Nstep,dt,thermostat_Nstep):
    Natom, _ = positions.shape
    velocities = np.zeros(positions.shape)
    accelerations = np.zeros(positions.shape)
    from tqdm import tqdm as tqdm_cs
    pos = []
    vel = []
    k_b = 8.61733e-5 # eV / K
        
    for it in tqdm_cs(range(Nstep)):
        
        if it % thermostat_Nstep == 0:
            velocities = andersen_thermostat(velocities,k_b*temperature)
        
        pos.append(positions)
        vel.append(velocities)
        
        positions,velocities,accelerations = move_velocity_verlet(positions,velocities,accelerations,
                                                                  dt,mass,r_m,epsilon,k_spring)
        
        
        
    return pos,vel

## trajectory visualiser

In [14]:
def traj_view(t):
    view=nv.show_asetraj(t, gui=True)
    view.clear_representations()
    view.add_ball_and_stick(aspectRatio=4)
    return view

In [19]:
def view_traj(pos,stride):
    from ase.visualize import view
    from nglview import show_asetraj,show_ase
    from ase import Atoms
    Natom,_ = pos[0].shape
    num = [1]*Natom
    #cell = pos[0].max()*10*np.eye(3)
    #frames = [Atoms(numbers=num,positions=pp,cell=cell,pbc=False) for pp in pos[::stride]]
    frames = [Atoms(numbers=num,positions=pp,pbc=False) for pp in pos[::stride]]
    view = show_asetraj(frames, gui=False)
    view.clear_representations()
    view.add_ball_and_stick(aspectRatio=4)
    return view

# run MD

In [16]:
from nglview import show_asetraj,show_ase

In [20]:
def make_linear_chain(N,r_m):
    r = N * r_m / (2*np.pi)
    n = np.arange(N)
    pos = np.asarray([r * np.cos(2*np.pi* n/N), r * np.sin(2*np.pi* n/N),np.zeros(N)]).T
    return pos.reshape((-1,3))
view = view_traj([make_linear_chain(N=10,r_m=1)],1)
view

NGLWidget()

In [21]:
def make_globular_chain(N,r_m,seed=10):
    np.random.seed(seed)
    pos = [np.zeros(3)]
    r = r_m
    for it in range(N):
        theta,phi = np.random.rand(2)
        pos.append(np.array(
                   [r * np.sin(theta*np.pi) * np.cos(phi*2*np.pi),
                    r * np.sin(theta*np.pi) * np.sin(phi*2*np.pi),
                    r * np.cos(theta*np.pi) ]
                    ) + pos[-1])
    return np.asarray(pos)
view_traj([make_globular_chain(N=10,r_m=1)],1)

NGLWidget()

In [22]:
epsilon = 1.
r_m = 1.
k_spring = 16.67 * epsilon / r_m**2
mass = 10.
print(np.pi / 6. * np.sqrt(2*epsilon / (mass* r_m**2)))
dt = 0.01
print(k_spring,dt)
initial_pos = make_linear_chain(100,r_m)

0.23416049103469086
16.67 0.01


In [23]:
T = 4.
pos,vel = MD_NVT_simulator(initial_pos,mass=mass,r_m=r_m,epsilon=epsilon,
                           k_spring=k_spring,temperature=T,
                           Nstep=10000,dt=dt,thermostat_Nstep=2000)

100%|██████████| 10000/10000 [00:39<00:00, 256.29it/s]


In [24]:
view_traj(pos,100)

NGLWidget(count=100)

In [84]:
pos,vel = MD_NVT_simulator(pos[-1],mass=mass,r_m=r_m,epsilon=epsilon,
                           k_spring=k_spring,temperature=T,
                           Nstep=10000,dt=dt,thermostat_Nstep=2000)

100%|██████████| 10000/10000 [01:20<00:00, 124.05it/s]


In [85]:
view_traj(pos,100)

In [35]:
initial_pos = np.array([[ 7.57791562,  6.05943954,  4.29313611],
       [ 9.68264401,  7.29780593,  6.16368313],
       [ 3.56309168,  3.4864184 ,  1.7344766 ],
       [ 4.34609445,  8.26733904,  1.73420634],
       [ 7.2692545 ,  1.6044641 ,  2.37728721],
       [ 9.95430504,  9.75743299,  1.97981315],
       [ 7.39230153,  7.12396028,  7.04031191],
       [ 3.4686374 ,  9.90738663,  6.84665733],
       [ 8.38584324,  6.83163809,  6.54699294],
       [ 1.40366425,  7.42216277,  0.10324773]])

In [37]:
initial_pos = np.array([
          [0.1947679907,        0.3306365642,        1.7069272101],
          [1.1592174250,       -1.1514615100,       -0.6254746298],
          [1.4851406793,       -0.0676273830,        0.9223060046],
         [-0.1498046416,        1.4425168343,       -0.9785553065],
          [1.4277261305,        0.3530265376,       -0.9475378022],
         [-0.6881246261,       -1.5737014419,       -0.3328844168],
         [-1.4277352637,       -0.3530034531,        0.9475270683],
         [ 0.6881257085,        1.5736904826,        0.3329032458],
         [-1.1592204530,        1.1514535263,        0.6254777879],
         [ 0.1498035273,       -1.4424985165,        0.9785685322],
         [-1.4851196066,        0.0676193562,       -0.9223231092],
         [-0.7057028384,        0.6207073550,       -1.4756523155],
         [-0.8745359533,        0.4648140463,        1.4422103492],
         [-0.9742077067,       -0.8837261792,       -1.1536019836],
         [-0.1947765396,       -0.3306358487,       -1.7069179299],
          [0.3759933035,       -1.7072373106,       -0.0694439840],
         [-1.7124296000,        0.3336352522,        0.1307959669],
         [ 0.9143159284,        1.3089975397,       -0.7151210582],
         [-0.3759920260,        1.7072300336,        0.0694634263],
         [ 1.7124281219,       -0.3336312342,       -0.1308207313],
         [-0.9143187026,       -1.3089785474,        0.7151290509],
         [ 0.9742085109,        0.8837023041,        1.1536069633],
         [ 0.7057104439,       -0.6206907639,        1.4756502961],
         [ 0.8745319670,       -0.4648127187,       -1.4422106957],
         [-1.1954804901,       -0.6171923123,       -0.1021449363],
         [ 0.0917363053,       -1.0144887859,       -0.8848410405],
         [ 0.9276243144,       -0.8836123311,        0.4234140820],
         [ 1.1954744473,        0.6171883800,        0.1021399054],
         [-0.9276176774,        0.8836123556,       -0.4234173533],
         [-0.3595942315,       -0.4863167551,        1.2061133825],
          [0.3595891589,        0.4863295901,       -1.2061152849],
         [-0.0917352078,        1.0144694592,        0.8848400639],
         [ 0.6410702480,       -0.1978633363,       -0.3898095439],
         [-0.4162942817,       -0.0651798741,       -0.6515502084],
         [ 0.1334019604,        0.7474406294,       -0.1600033264],
         [-0.6410732823,        0.1978593218,        0.3898012337],
         [ 0.4162968444,        0.0651733322,        0.6515490914],
         [-0.1333998872,       -0.7474445984,        0.1600019961],
])

In [38]:
view_traj([initial_pos],1)